<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Simple_Agents_SDK_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Einfache Agents SDK Demo

In [1]:
!pip install -qU openai-agents==0.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 7.5 MB/s eta 0:00:00


In [15]:
import os
import json
from agents import Agent, Runner, function_tool, set_tracing_export_api_key

# Parameter
buchungsart = "Reisekosten" # Tankkosten, Spesen
betrag = 122.00

# OpenAI API Key aus Colab Secret auslesen
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

set_tracing_export_api_key(os.environ.get('OPENAI_API_KEY'))

# Funktionen definieren
@function_tool
def record_travel_costs(amount: float):
    return "Reisekosten wurden verbucht."

@function_tool
def record_fuel_costs(amount: float):
    return "Tankkosten wurden verbucht."

@function_tool
def record_expenses(amount: float):
    return "Spesen wurden verbucht."

# Agents definieren
travel_cost_agent = Agent(
    name="Travel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Reisekosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_travel_costs],
)
fuel_cost_agent = Agent(
    name="Fuel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Tankkosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_fuel_costs],
)
expenses_agent = Agent(
    name="Expenses Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Spesenrechnungen. Nutze dafür die bereitgestellten Tool.",
    tools=[record_expenses],
)

# Triage Agent konfigurieren
triage_agent = Agent(
    name="Triage Agent",
    instructions="Du routest den Benutzer zu einem passenden Agent. Stelle bitte keine Rückfragen an den Benutzer.",
    handoffs=[fuel_cost_agent, travel_cost_agent, expenses_agent]
)

# Prompt zusammenbauen
user_query = f"Verbuche bitte meine {buchungsart} von €{betrag:.2f}."
print("User:", user_query)

# Agents ausführen
result = await Runner.run(
    starting_agent=triage_agent,
    input=user_query#"Stelle ein einfaches Mittagsmenü zusammen."
)
output = result.final_output

print(output)


User: Verbuche bitte meine Reisekosten von €122.00.
Deine Reisekosten von €122.00 wurden erfolgreich verbucht.
